In [2]:
%%shell
echo "Hello from the shell!"
pwd

Hello from the shell!
/content


In [6]:
# whisper for speech recognition
# OPENAI supported speech to text model, but we will not use this one due to some error I cant figure out
# i will use whisper model but the one hosted on the hugging face transformers library. This will allow to be create pipelines for model analysis and transcription
import whisper
#vosk is a open source , offline speech recognition library( so does not need internet and is lightweight)
# Vosk uses the kaldi engine, integrating speech models into apps, and comes with pretrained models
#kaldi is a  toolkit for speech recognit and dev. provides algos for building asr systems
#kaldirecognizer is an instance within vosk framework for doing the transcription of audio
from vosk import Model as VoskModel, KaldiRecognizer
# for opening, reading and writing wav files
import wave
#pyTorch for wav2vev2, helpful with these neural network models im using
import torch
#wav2vev2 model and tokenizer from HuggingFace
# hugging face is central repo for pretained models, hugging face provides the wav2vec2 model and the tokenizer
# Wav2Vec2 is speech to text model and self supervised speech model  that learns patterns from raw audio.pretrained on lots of unlabeled audio data, covering various speeches.
#this model is used with CTC( connectionist temporal classification) . CTC is extra layer to the model , where model learns to predict seq of chars from the extracted features.
# the tokenizer converts model output into seq of tokens ( characters or subwords). Decodes output into readable text.
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
#for word error rate , lower the better
from jiwer import wer
import time
import numpy as np
# i will test 3 asr models : whisperAI, vosk and wav2vev2. for each 3 metrics are measured : WER ( how diff models transripts is from ground truth)
# 0.0 means perfect transcription. HigherWER --> worse . Lower wer --> accurate model . WER = (number of insertions + deletions + substitutions) / (number of words in ground truth)
#time is measured ( how long model took to transcribe audo file from start to finish  of prediction process)
# transcription text metric : acc string of text produced by model for audio input.




Hello, my name is Amrish. I'm currently a fourth year studying computer science at North carolina state university. Two years ago in 2023, AI and machine learning got super important in tech, healthcare, and even education. In this project, I will be trying to prototype a ASR Model that begins with English to English conversion. Then I will extend the model to support Tamil to English conversion. Finally I will try Sourashtra to English. Saurashtrians are a linguistic and cultural community in Tamil Nadu, India, originally from the Saurashtra region of Gujarat, who speak the Indo-Aryan Saurashtra language.

This text may be good because it has a good mix of numbers, commas, case sensitiive names and grammar a good ASR model should know how to transcribe.


In [7]:

#testing asr model accuracy
from google.colab import files
print("Upload an .wav audio file")
uploaded = files.upload()
#turning the uploaded files ( dict) into numpy list( the whisper pipelines expect numpy array as input), and picks first one ( assuming you uploaded 1 file)
audio_file = list(uploaded.keys())
if len(audio_file) > 0 :
  audio_path = audio_file[0]
else :
  print("No file uploaded")
  audio_path = " "
print(audio_path)

#get ground truth transcript so that we can compare model output with acc results to achieve metrics
ground_truth = input("paste actual transcript for the audio: ")
#clean up spaces and lower case
ground_truth = ground_truth.strip().lower()
print("Ground truth:", ground_truth)



Upload an .wav audio file


Saving ASRAudioTesting.wav to ASRAudioTesting (1).wav
ASRAudioTesting (1).wav
paste actual transcript for the audio: Hello, my name is Amrish. I'm currently a fourth year studying computer science at North carolina state university. Two years ago in 2023, AI and machine learning got super important in tech, healthcare, and even education. In this project, I will be trying to prototype a ASR Model that begins with English to English conversion. Then I will extend the model to support Tamil to English conversion. Finally I will try Sourashtra to English. Saurashtrians are a linguistic and cultural community in Tamil Nadu, India, originally from the Saurashtra region of Gujarat, who speak the Indo-Aryan Saurashtra language.
Ground truth: hello, my name is amrish. i'm currently a fourth year studying computer science at north carolina state university. two years ago in 2023, ai and machine learning got super important in tech, healthcare, and even education. in this project, i will be tryi

In [8]:
#Testing  whisperAI ASR model with timer
print("testing whisperAI model")
#loading pre trained whisper model - base model (speed and accurate)
import whisper
whisper_model = whisper.load_model("base")
start_time = time.time()
#transcribe text
whisper_output = whisper_model.transcribe(audio_path)
# result is a dict, this is getting the value ( the transcribed text) from the 'text' key having the transcript
# gets the main transcribed text
transcribed_text = whisper_output["text"]
#if "text " key not in dict, the transcribed_text set to empty string
if ("text" not in whisper_output):
  transcribed_text = " "

#removes leading spaces and converts to lowercase.
transcribed_text = transcribed_text.strip().lower()
end_time = time.time()
elapsed_time = end_time - start_time
#calc wer rate against ground truth
whisper_wer = wer(ground_truth, transcribed_text)
print("\n[Whisper]")
print("WER:", round(whisper_wer, 2))
print("Time:", round(elapsed_time, 2), "seconds")
print("Transcription:", transcribed_text)

testing whisperAI model


AttributeError: module 'whisper' has no attribute 'load_model'

In [9]:
#testing wh#isper ai model (via hugging face)
#pipeline function gives easy acess to pre trained models hosted on hugging face
from transformers import pipeline
import time

#loading whisper ai using hugging face pipeline, "automatic-speech-recognition" is task name, second pam is the model we want pipeline to use ,
#third param means output will include time segments for each phrase ( allows for long form generation )
whisper_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-base", return_timestamps=True)

start_time = time.time()
# transcribe the audio file, whisperpipeline called with audio path returns dict with "text" holding transcription data
transcribed_text = whisper_pipeline(audio_path)["text"]
end_time = time.time()
elapsed_time = end_time - start_time

# clean up transcribed text
transcribed_text = transcribed_text.strip().lower()

# Calculate WER
# Ensure ground_truth variable is available from previous cells
whisper_wer = wer(ground_truth, transcribed_text)
print("\n[Whisper (transformers)]")
print("WER:", round(whisper_wer, 2))
print("Time:", round(elapsed_time, 2), "seconds")
print("Transcription:", transcribed_text)

#diff between this whisper model and previous is that in previous, i used the original OpenAI Whisper library
# using pip install openai-whisper. Now I want to extend this model to tamil again to see its accuracy on Tamil



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.



[Whisper (transformers)]
WER: 0.05
Time: 47.8 seconds
Transcription: hello, my name is amrish. i'm currently a fourth year studying computer science at north carolina state university. two years ago in 2023, ai and machine learning got super important in tech, healthcare, and even education. in this project, i will be trying to prototype a asr model that begins with english to english conversion. then i will extend the model to support tamil to english conversion. finally, i will try sarashtra to english. sarashtrians are a linguistic and cultural community in tamil nadu, india, originally from the sarashtra region of gujarat, who speak the indo-aryan sarashtra language.


the tamil to english grouth truth
	அவனை எரிக்கும் அதே நெருப்பு என்னையும் எரிக்கட்டும் என்று விம்மி விம்மி அழுது அலுத்துக்கூறினான்.	கீழே விழுந்துவிடாமல் தம்மை அடிக்கடி தூக்கி நிறுத்திக் கொண்ட மனிதசாதி கெட்டதைப் போல செடிகள் உலகம் கெடவில்லை.
  to english reads : "Let the same fire that burns him burn me too," he cried, wearily.
The world of plants is not as bad as the human race, which has repeatedly picked itself up without falling down.

In [ ]:
#attempting to use tamil for whisper ai
#found tamil speech audio on Mozilla common voice, 62.7 mb --> TAMIL TO TAMIL ASR
#short audio , 17 secs of audio the truth text is above this code block
#takes tamil audio file , transcribes to tamil text using whisper and prints accuracy metrics
from google.colab import files
from transformers import pipeline
from jiwer import wer
import time
import re # Import the re module


print("Upload an .wav audio file")
uploadedT = files.upload()
#turning the uploaded files ( dict) into list, and picks first one ( assuming you uploaded 1 file)
audio_fileT = list(uploadedT.keys())
if len(audio_fileT) > 0 : # Corrected variable name here
  tamil_path = audio_fileT[0]
else :
  print("No file uploaded")
  tamil_path = " "
print(tamil_path)

#extra step to clean texts, removing punc, normalize spacing , lower case
def normalize_text(text):
  text = text.lower()
  # Remove punctuation
  text = re.sub(r'[^\w\s\u0B80-\u0BFF]', '', text)
  # Normalize whitespace
  text = re.sub(r'\s+', ' ', text)
  return text.strip()


#get ground truth transcript so that we can compare model output with acc results to achieve metrics
ground_truthT = input("paste actual tamil transcript for the audio: ")
#clean up spaces and lower case
ground_truthT = normalize_text(ground_truthT)
print("Ground truth:", ground_truthT)

#load whisper pipeline for asr tamil to tamil
#whisper auto detects language and transcribes to original ( tamil in this case)
whisper_pipelineTamil = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base", #better accuracy
    return_timestamps=True,
    generate_kwargs={"language": "ta", "task": "transcribe"} # pass language and task in generate_kwargs, telling to transcribe tamil audio to tamil text by defualt
)
print("Transcribing tamil audio")
start = time.time()
result = whisper_pipelineTamil(tamil_path)
transcribed_text = normalize_text(result["text"])
end= time.time()
tamilTime = end- start

tamil_wer = wer(ground_truthT, transcribed_text)
print("\n[Whisper Tamil → Tamil]")
print("WER:", round(tamil_wer, 2))
print("Time:", round(tamilTime, 2), "seconds")
print("Transcription:", transcribed_text)

#interesting to see that even though both translatoins are diff, the wer is still the same , both .91
# wer blind to meaning - only counts word - level matches , not if sentence makes sense or closer meaning
# from chat ai - if both outputs are similarly incorrect (i.e., require lots of edits), you can get the same (or similar) high WER values—even if the content and words are different
# with google translate , I can try BLEU( bilingual evaluation understudy)- extra metric  as it measures overlap in phrases and meaning better


Upload an .wav audio file


Saving tamilExtension.wav to tamilExtension (14).wav
tamilExtension (14).wav
paste actual tamil transcript for the audio: அவனை எரிக்கும் அதே நெருப்பு என்னையும் எரிக்கட்டும் என்று விம்மி விம்மி அழுது அலுத்துக்கூறினான். கீழே விழுந்துவிடாமல் தம்மை அடிக்கடி தூக்கி நிறுத்திக் கொண்ட மனிதசாதி கெட்டதைப் போல செடிகள் உலகம் கெடவில்லை.
Ground truth: அவனை எரிக்கும் அதே நெருப்பு என்னையும் எரிக்கட்டும் என்று விம்மி விம்மி அழுது அலுத்துக்கூறினான் கீழே விழுந்துவிடாமல் தம்மை அடிக்கடி தூக்கி நிறுத்திக் கொண்ட மனிதசாதி கெட்டதைப் போல செடிகள் உலகம் கெடவில்லை


Device set to use cpu


Transcribing tamil audio

[Whisper Tamil → Tamil]
WER: 1.08
Time: 35.25 seconds
Transcription: கிழே விழந்து விடமல் தம்மை அடிக் கடி தூகே நிருத்திக்கொண்டு மனிதா சாதிக் கட்டதை போல செடிக்கல் உலகம் கடவில்லை அவனையிருக்கும் அதே நிருப்பு என்னையும் விரிக்கட்டும் என்று விம்பி விம்பி அழுத்து அழுத்து கூறினான்


In [ ]:
#translate tamil transcription to english
# this way translates the transcribed tamil to englsih using hugging face pipeline
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
print("\nTranslating Tamil transcription to English...")
#this translation pipeline will translate the transcripted tamil text above to english
translation_pipeline = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M", # this is the multilangual translation model
    src_lang="tam_Taml", #source lang
    tgt_lang="eng_Latn" # output lang - target
)
start_trans = time.time()
# tamil text is passed in as input and pipeline processes this text using the pretrained translation model from facebook
# [0] since pipeline can return list of possible translations, 0 returns most likely translation from results
translated_english = translation_pipeline(transcribed_text)[0]['translation_text']
end_trans = time.time()
translation_time = end_trans - start_trans

print("[Tamil → English Translation]")
print("Translation:", translated_english)
print("Translation Time:", round(translation_time, 2), "seconds")

#computing wer on englsih ground truth as well
ground_truthE = input(" Paste the actual English translation for the audio (for WER): ").strip().lower()
def normalize_en(text):
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s\u0B80-\u0BFF]', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()
    #text = text.lower()
    #text = re.sub(r'[^\w\s]', '', text)
    #text = re.sub(r'\s+', ' ', text)
    #return text.strip()
#running our truth text and translated text through a normalization algo so that wer comparison is possible, and that puncuation and extra chars don't mess with the asr model
ground_truthE = normalize_en(ground_truthE)
translated_english_norm = normalize_en(translated_english)
english_wer = wer(ground_truthE, translated_english_norm)
print("English WER:", round(english_wer, 2))





Translating Tamil transcription to English...


Device set to use cpu


[Tamil → English Translation]
Translation: Wimpy Wimpy said, "The flowering world doesn't pass away like a human race without falling to the ground".
Translation Time: 22.35 seconds
 Paste the actual English translation for the audio (for WER): "Let the same fire that burns him burn me too," he cried, wearily. The world of plants is not as bad as the human race, which has repeatedly picked itself up without falling down.
English WER: 0.91


In [ ]:
#this way does tamil to english translation using google trans library,instead of hugging face library
from googletrans import Translator
translator =  Translator()
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

print("Translating Tamil transcription to English...")
start_trans = time.time()
#using google translate API to translate Tamil to english, #src is tamil and dest is english
#translate method returns Translation obj. by doing .text on this object, we can the translate text as a string
translated_englishGT = translator.translate(transcribed_text, src='ta', dest='en').text
end_trans = time.time()
translation_time = end_trans - start_trans

print("[Tamil → English Translation]")
print("Translation:", translated_english)
ground_truthEGT = input("\n(Optional) Paste the actual English translation for the audio (for WER): ").strip().lower()
#normalization algo as before
def normalize_en(text):
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s\u0B80-\u0BFF]', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()
    #text = text.lower()
    #text = re.sub(r'[^\w\s]', '', text)
    #text = re.sub(r'\s+', ' ', text)
    #return text.strip()

#computing wer between the translated english and the truth english conversion of our tamil audio ( same as before )
ground_truthEGT = normalize_en(ground_truthEGT)
translated_english_normGT = normalize_en(translated_englishGT)
english_wer = wer(ground_truthE, translated_english_normGT)
print("English WER:", round(english_wer, 2))








Translating Tamil transcription to English...
[Tamil → English Translation]
Translation: The Sedical world is not like a man who is a caste, and he will not fall on the east, and he will spread the same seredenness to me.

(Optional) Paste the actual English translation for the audio (for WER): "Let the same fire that burns him burn me too," he cried, wearily. The world of plants is not as bad as the human race, which has repeatedly picked itself up without falling down.
English WER: 0.91


In [ ]:
#fine tune whisper , above I just tested with a small set of input audios
# i should upload a plethora of audio files in tamil to train and potentially tune, helping with domain or accent adaption
#creating a dataset with columns audio and transcript in tamil, allowing me to create parallel Tamil English datasets
# from this file, I can also populate hindi , english and eventually sourashtra translations of the audio
import pandas as pd
import os
from pydub import AudioSegment # audio conversion
from deep_translator import GoogleTranslator # Import GoogleTranslator from deep_translator

#reading in the validated.tsv file ( this file has meta deta about the audio files. I have linked these audio files in the clips directory here)
# using the tsv file and its cols (the path and transcript ), I will make output df , so I can prepare the resulting dataframe for model training

df = pd.read_csv("validated.tsv", sep="\t")
if df is not None:
    # check if the 'clips' directory exists
    clips_dir = "clips"
    if not os.path.exists(clips_dir):
        print(f"Directory not found: {clips_dir}")
    else:
        #output dir for wavs
        wav_clips_dir = "wav_clips"
        os.makedirs(wav_clips_dir, exist_ok=True)
        #convert mp3 to wav , create lists for dataset, this will hold full paths and transcripts for each tamil audio
        audio_paths = []
        transcripts = [] # tamil
        engTranscripts = [] # english
        hindiTranscripts = [] # hindi

        # Initialize the translator outside the loop for efficiency
        translator_en = GoogleTranslator(source='ta', target='en')
        translator_hi = GoogleTranslator(source='ta', target='hi')

        # loop will go through each entry in the tsv file and extract the path name
        for index, row in df.iterrows():
            mp3_file = os.path.join(clips_dir, row["path"])
            #construct input mp3 and output is wav
            wav_file = os.path.join(wav_clips_dir, row["path"].replace(".mp3", ".wav"))

            #convert only if wav does not already exist
            if not os.path.exists(wav_file):
                try:
                    #load the mp3 , set sample rate ( covert to mono) and export the file as wav
                    audio = AudioSegment.from_file(mp3_file)
                    audio = audio.set_frame_rate(16000).set_channels(1)  # 16kHz mono
                    audio.export(wav_file, format="wav")
                except FileNotFoundError:
                     print(f"Mp3 file not found: {mp3_file}")
                     continue # skip this file and continue with the next
                except Exception as e:
                    print(f"Error converting {mp3_file} to WAV: {e}")
                    continue # skip on other errors too

            # i am populating the audio path and transcripts list
            audio_paths.append(wav_file)
            #row will be dict ( keys represent the column name)and sentence key holds the sentence value for that particular audio file
            transcripts.append(row["sentence"])

            #translate to english and hindi using deep_translator
            try:
                eng_translation = translator_en.translate(row["sentence"])
                hindi_translation = translator_hi.translate(row["sentence"])
                engTranscripts.append(eng_translation)
                hindiTranscripts.append(hindi_translation)
            except Exception as e:
                print(f"Error translating sentence for row {index}: {e}")
                engTranscripts.append("") # Append empty string on error
                hindiTranscripts.append("") # Append empty string on error


        #now we can build dataframe and save as dataset with tamil audio and its tamil translation as specified in validation.tsv
        dataset_df = pd.DataFrame({
            "audio": audio_paths,
            "transcript_Tamil": transcripts,
            "transcript_English": engTranscripts,
            "transcript_Hindi": hindiTranscripts
        })

        dataset_df.to_csv("tamil_asr_dataset.csv", index=False)
        print("✅ Dataset saved as tamil_asr_dataset.csv")
        print(dataset_df.head())

✅ Dataset saved as tamil_asr_dataset.csv
                                    audio  \
0  wav_clips/common_voice_ta_37342302.wav   
1  wav_clips/common_voice_ta_37311038.wav   
2  wav_clips/common_voice_ta_37311039.wav   
3  wav_clips/common_voice_ta_37311133.wav   
4  wav_clips/common_voice_ta_37318886.wav   

                                    transcript_Tamil  \
0  அதன்படி மறுநாள் காலை சென்னைக்கு வந்து சேர்ந்தோம்.   
1  அவனை எரிக்கும் அதே நெருப்பு என்னையும் எரிக்கட்...   
2  கீழே விழுந்துவிடாமல் தம்மை அடிக்கடி தூக்கி நிற...   
3  இங்ஙனம் செயற்படுத்த இயலாவண்ணம் நிற்போருக்கு யா...   
4  காரைத்தான் அப்பா காலையிலே எடுத்துக் கொண்டு போவ...   

                                  transcript_English  \
0  Accordingly we arrived in Chennai the next mor...   
1  Wimmy Wimmy cried and said, "Let me burn the s...   
2  The world is not as bad as the humanitarian ev...   
3  Who is the companion of the people who are una...   
4  It was only then that I remembered that the ca...   

              

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict, Audio
#Dataset- fund data struct in hugging face datasets library. a single collection fo data samples ( aduio files,transcripts,computer vision, etc).
#DatasetDict - dict like object used to hold multiple datast objects. after splitting dataset into test and training, we have
# a dataset dict using the train and test dataset.
#Audio - when casting audio file path to Audio feature, datasets lib knows how to load audio file from path, decode it and store as numerical arrray ( numpy or pytorch tensor)
#resamples audio to specific rate, important here since whisper wants input sample rate as 16000 Hz and mono

#https://huggingface.co/blog/fine-tune-whisper  ---> this link was what i used to help with fine tuning of whisper ai model
# in this link, the author used a big hindi dataset for training, but I'm using something much smaller for time issues( cant wait 5 hours for model to train as shown in article)
# the point of this code chunk is to run my tamil parallel csv ( the one with audio, tamil, english, hindi)
# against the base open ai whisper asr model.

# load  your custom CSV
df = pd.read_csv("tamil_asr_dataset.csv")
df = df.rename(columns={"transcript_Tamil": "sentence"})  # standardizing col name for consistency

# convert the pandas dataframe to hugging face dataset
dataset = Dataset.from_pandas(df)
#here since input audio is sampled at 48khz, we need to downsample to 16, cast_column signals to dataaset to resample audio files on the fly as they are loaded
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000)) #telling datasets lib to treat audio col as
#audio data. when column assessed later, lib will load audio file from path in each row, decode it , and reample to sample rate of 16000 hZ
# Audio feature stores processed audio data as numerical array within dict struct.

# Split into train/test
ds = dataset.train_test_split(test_size=0.3, seed=42)
print(ds)

#this is the baseline wer before fine tuning my asr model using the tamil_asr_dataset.csv
#pipeline is way to use pre trianed models for tasks, like ASR , without model loading
#pipelines are objects that abstract complex code from transformers lib , making it easy to work with asr models
from transformers import pipeline
import evaluate ##evaluate lib provides standard metrics for eval these models( WER)

# Load the original pre-trained Whisper pipeline (no fine-tuning) . uses the whisper-small model and its tokenizer
#pipeline take audio input and produce text transcription using pre trained weights of model
asr_pipe = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-small",
    tokenizer="openai/whisper-small"
)

# Run predictions on your test set
#two lists to store models predicted results and ground truth transcription
predictions = []
references = []
#for loop that will run the model on the specified test data in my dataset
# target value is the transcript_Tamil and the prediction by the model is the predicted value
for item in ds["test"]:
    # extracts audio data as numerical array from audio feature , datasetslib, because you cast column to Audio, has already handled loading and resampling the audio to 16kHz
    audio_input = item["audio"]["array"] # acc array containing audio samples is under the key "array" within "audio" feature, numpy array has the audio waveform data
    #extracts truth text for this particular audio
    reference_text = item["sentence"]
    # pretrained whisper model loaded via the asr_pipeline pipleine processes audio input. performs asr and returns result
    result = asr_pipe(audio_input)
    #returned result is a dict, we acess the text key to get transcribed text for that audio
    predicted_text = result["text"]
    #append prediction and reference to lists for metric comparsion
    predictions.append(predicted_text)
    references.append(reference_text)

# loading  WER metric from evaluate library
wer_metric = evaluate.load("wer")
#compares each predicted setence to its corresponding ref sentence , using the equation
# (insertions, deletions, or substitutions) needed to change prediction to be = to reference, divided by #of words in ref
baseline_wer = 100 * wer_metric.compute(predictions=predictions, references=references)
print("Baseline WER (no fine-tuning):", baseline_wer)






DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'transcript_English', 'transcript_Hindi'],
        num_rows: 9
    })
    test: Dataset({
        features: ['audio', 'sentence', 'transcript_English', 'transcript_Hindi'],
        num_rows: 4
    })
})


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Baseline WER (no fine-tuning): 58.82352941176471


In [ ]:
# whisper is Transformer based encoder-decoder model( seq to seq model). Maps a seq of audio spectrogram features to a sequence of text tokens
#1. raw audio inputs are converted to log-Mel spectogram by action of feature extractor. The Transformer encoder then encodes the spectrogram to form a sequence of encoder hidden states
# Finally, the decoder autoregressively predicts text tokens, conditional on both the previous tokens and the encoder hidden states.

#in this article, it says 680,000 hours of labelled pre-training data, Whisper models demonstrate a strong ability to generalise to many datasets and domain
# these pre trained checkpoints have 3% wer on the test-clean subset of LibriSpeech ASR and a new state-of-the-art on TED-LIUM with 4.7% WER (c.f. Table 8 of the Whisper paper)
# the asr knowledge acquired thru whisper during pre trinaing can be used with my low level resource lang: sourashtra -->thru fine tuning, i can adapt these pre-trained checkpoints

#whisper is pre trained and fine tuned ;trained  to correctly classify the target text token from a pre-defined vocabulary of text tokens
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

#transformers to load and train our whisper model
# soundfile to pre process audio files, evaluate and jiwer for performance, tensorboard to log metrics.

# dataset was split in training and testing in above chunk

#asr pipeline is decomposed into 3 parts : a feature extractor which pre processes raw audio inputs
#, model which performs the seq to seq mapping , tokenizer which post processes the model outputs to text format

#WhisperFeatureExtractor: speech is a 1d dim array that varies with time.  value of array at time setp is the signals amplitude.
#speech is cont, so it has inf number of amptlitude values. interval in whcih we sample our audio is known as
# sampling rate and is measured in hz. whisper expects audio inputs with rate of 16khz.
#extractor has two operations: first pads/truncuates batch of audio samples such that all samples have input len of 30s, samples shroter is padded with 0, longer is truncuated
#second operation is that extractor coverts padded audio arrays to log mel spectorgrams( spectograms are visual reps of freq of a signal)
#the spectogram is the what the whsiper model encoder expects as input
#Spectrogram (input to Encoder) ➡️ Encoder Processing ➡️ Encoder Hidden States ➡️ Cross-Attention Mechanism in Decoder ➡️ Decoder Processing (using encoder states and previous tokens) ➡️ Predicted Text Token (output of Decoder)
#WhisperTokenizer
#Whisper uses BPE (Btye - pair encoding) for tokenization. This method breaks down words into smaller subword units,
#allowing the tokenizer to handle rare or out-of-vocabulary words effectively
#whisper model outputs text tokens that indicate index of predicted text among the dict of vocab terms( model has list of all possible tokens it can use) --> the integer id represnets the index of predicted result in the tamil vocab
# tokenizer maps a seq of text index ids to acc text string : e.g. [1169, 3797, 3332] -> "the cat sat") --> converts text like tamil trasncirpt into seq of numerical token ids that the whisper model decoeder understans and vice versa

#normally, when we  use encoder only models for asr, we decode using Connectionist Temporal Classification (CTC).
# whisper tokenizer already pre trained on transcriptions of 96 pre training lang- including tamil/hindi. we load the
# tokenizer by specifying target language and task.
model_name = "openai/whisper-small"   # or base/medium if you want
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
#these  arguments inform the tokenizer to prefix the language and task tokens to the start of encoded label sequences:
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="Tamil", task="transcribe")
# to simplify using feature extractor and tokenizer, im wrapping both into single WhisperProcessor class. we need to keep track of two objects during training: processor and model
#processor used to simplify the steps to prepare audio for the model, and decode the predicted ID’s back into text.
#use processor to handle audio ninput and text labels
processor = WhisperProcessor.from_pretrained(model_name, language="Tamil", task="transcribe")
#encoding --> tokenizer appends special tokens to start /end of seq, such as lang token, task token
#=-- processes entire input spectogram into representation that captures  phonetic and linguistic information--> produces seq of encoder hidden states( summary of audion content )
#decoding --> when decoding label ids, we can skip these speical tokens, allowing us to return string of original input form
#-- decoder takes encoder hidden states and seq of previsouly predicted text tokens as input --> generates output text transcription, tok by tok
#predidicts the next token based on the input audion , we need predicted text tokens to to understand the text decoder  has already generated
# decoeder uses cross attention to use the info from encoder hidden states , at teach step while its making a token,
# the decoder looks at diff parts of encoder input( the hidden states and seq of token ids it has already generated )to find info relevant to predicting current text toek ( decoder can hear the relev parts of audion to create transcription)
# then tokenizer maps seq of ids back to human readable text
# since the direct output of decoder in model is seq of numerical token IDS( each toekn in the vocab assigned a uniqe int id)
# converseley, when tokenizer converts text into numerical form, it breaks text into tokens and replaces each token with id

#preparing data ready for model
#load and resample audio data , we use feature extractor to compute logmel spectogram input features from 1-dim audio file
# seems like each sample of the audio in thee file is the batch
def prepare_dataset(batch):
    audio = batch["audio"]
    #calling Extractor with audio array and its sampling rate.
    # feature_extractor returns dict , and acc spectogram features are stored under input_features key, since extractor is procesisng
    # a batch, input_features is a list, and [0] acesses features for the first in the batch( in our case we only one sample in the batch processed func ATM)
    #resulting spectogram features added to the batch dict under new key "input_features"
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids( numerical token ids which model decoder and training process expect for target labels)
    # we are acessing ground truth for curent sample, .input_ids accesses seq of token ids generated by tokenizer and resulting token id added to batch dict
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch
# this applies prepare_dataset function to dataset ds( the training and testing splits), applying to each batch or sample of data
#remove_columns tells map to remove original columns from ds after function has been applied.
#audio, sentence no longer needed , since we transformed to input_features and labels columns
ds = ds.map(prepare_dataset, remove_columns=ds["train"].column_names)

#training and evaluation, The Trainer class helps us here: provides an API for feature- completing in pytorch
#goes hand in hand with TrainingArguments class, which gives wide range of options to customize how a model is trained

# we first have to load pre trained checkpoint and configure it correctly for training, define a data collator( this takes our pre procssed data and prepares pytorch tensors ready for model)
#since we want wer metric, we need a compute_metric function that handles this computation, we define training args( used by trainer in making training scheduel)
# we then eval on test data to see how accuratly we have trained it to transcribe speech to tamil
import torch

#pretrained weights are the learned params (weights and biases) of a Neural network that has already been trained on large datasets
from transformers import WhisperForConditionalGeneration # this class used when conditional generation tasks like ASR where model creates text conditional on the audion input
model = WhisperForConditionalGeneration.from_pretrained(model_name) # laods model arch and its pretrained weightsfrom hugging face hub
model.generation_config.language = "tamil" #tells model when generating text , should aim to produce tamil text,helping model uses its internal knowledge about the lang
model.generation_config.task = "transcribe" # tells model that task is to transcribe audio ( speech to text in same lang)
model.generation_config.forced_decoder_ids = None #forces decoder to generate specific tokens at start of generated seq , None means no tokens are forcced, allowing decoder to decide starting seqeunce
#based on input audio, lan and task

#data collator take list of indiv data samples from dataset and combine them into single batch, ensuring all seq in batch have same len (padding)
#data collator for a seq-seq speech model is unique in that treats the input_features and labels independently: the input_features must be handled by the feature extractor and the labels by the tokenizer.
# input features already padded to 30s, we just have to convert log mel spectogram to Pytorch tensor.
#labels are unpadded - first pad seq to max len in batch using tokenizer .pad method. padding tokens replaced with -100, tokens are not taken into account when computing loss
#then cut start of transcript token from the beginning of the label sequence as we append it later during training.
class DataCollatorSpeechSeq2SeqWithPadding:
  #class expects a processor object, and int repping token id that signals start of a seq for decoder
    processor: any
    decoder_start_token_id: int
    #constuctor for the class
    def __init__(self, processor, decoder_start_token_id):
        self.processor = processor
        self.decoder_start_token_id = decoder_start_token_id
    #When Trainer needs to create batch of data, it will call this method and pass list of data samples ( dicts, each represnting one sample) to the features argument
    #this method then processes features to create padded batch
    def __call__(self, features):
        #creates a list of dict, where each dict contains the spectogram features ("input_features") for one sample from input features list
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        #after preparing data for padding by feature extractor, we use the pad method of the feature_extractor acessed via the processor to pad
        #input_features so that all spectograms in batch have same len, return_tensors = "pt" tells to return res as Pytorch tensors
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt") # padded batch of spectograms are stored in batch dict
        #get list of dict having tokenized label ids (labels) for each sample, prepping for padding
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        #uses pad method of tokenizer to pad label_features so that all label sequences have same len, result as pytorch tensor
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        #replaces padding tokens (which are at positions where the attention_mask is not 1) with -100 , these are igored
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        #hthis checks to see if first token in all label sequences in the batch is the decoder start tokem
        #if it is remove the token, because the decoder implicity starts with start token and you dont need to predict it again as first label
        #we can cut start of transcript token from beginning of label seq as we append it later during training
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        # batch dict has padded input features and labels
        return batch
#creates instacne of the above class.Passes processor object and token ID that signifies begining of a seq for the decoder
# this object is then given to hugging face trainer to handle batch creation during training
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=tokenizer.bos_token_id
)


#using wer for metric, need func that takes our model predictions and retursn wer metric.
import evaluate
metric = evaluate.load("wer")
#replaces -100 with the pad_token id in the label_ids( undoing the step we applied in the data collator to ignore padded tokens correclty in loss)

def compute_metrics(pred):

    pred_ids = pred.predictions
    label_ids = pred.label_ids
    #during data preparation with data collaotr, padding tokens in labels converted to -100 so loss func ignore it
    #now, we decode lables back into text for we so we need to replace -100 values back with the acc padding token id used by tokenizer
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    #converting predicted and label ids into list strings ; skip_spcial_tokens= true is used to ignore special tokens ( padding tokens, start/end tokens) during decoding process, so you just get transcribed text
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    #comparing predicted string vs the reference string( truth variable)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    #retuns dict containing calc wer, Trainer wants metric to be returned in dict format
    return {"wer": wer}
#Trainer class abstracts complexities of machine larning training loop, instead of manually writing code to
#Iterating through epochs Processing batches of data Performing forward passes through the model Calculating the loss,
#Trainer takes care of all this. You config the trainer by providing it with model you want to train, training args, and the test/trianing datasets, and func to compute metrics

# in final step, we define all params related to training;  init Seq2SeqTrainer with theseqence to sequence model, data and arguments, starting the training and saving trained model/processor
from transformers import Seq2SeqTrainingArguments
# various hyperparams, the params are xplained in the article
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tamil-finetuned",
    per_device_train_batch_size=2,          # Small batch size, fits on most GPUs/Colab
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,          # No need for accumulation with small batch
    num_train_epochs=10,                    # More epochs since you have so little data
    learning_rate=5e-5,                     # Slightly higher LR to learn quickly
    save_steps=10,                          # Save frequently (not really needed here)
    eval_steps=5,                           # Evaluate frequently
    logging_steps=1,                        # Log every step for visibility
    predict_with_generate=True,
    generation_max_length=225,
    fp16=False,                             # Use True only if you have a GPU with FP16
    eval_strategy="steps",
    save_strategy="no",                     # Don't waste time saving checkpoints
    report_to=[],                           # No TensorBoard/W&B unless you want logs
    load_best_model_at_end=False,           # Not needed for tiny debug run
    metric_for_best_model="wer",
    greater_is_better=False,
    # Remove push_to_hub, warmup_steps, gradient_checkpointing for simplicity
)
#Imports Trainer class for seq- seq models
from transformers import Seq2SeqTrainer
# creating instance of Trainer, providing the training arguments, whisper model instance you loaded earlier,
#training/eval dataset, the data collator you defined to prepare batches, func to define wer, tokenizer object
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,  #looks wrong but that what is shown on article, --> should be changed to processor.tokenizer
)
#starts the training process process. Trainer will run the training loop for specified number of epochs or steps
trainer.train()
#trainer uses the data collator to handle batch creation and padding

trainer.save_model("./whisper-tamil-finetuned")
processor.save_pretrained("./whisper-tamil-finetuned")

#weird that fine tuning does not reduce the wer rate. we probably need more data to be trained on , but I can't wait hours


Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/tmp/ipython-input-14-215312517.py:184: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Wer
5,1.178900,0.927745,67.647059
10,0.257400,0.724700,147.058824
15,0.249900,0.673844,67.647059
20,0.236100,0.703254,79.411765
25,0.100800,0.680304,76.470588
30,0.087000,0.674514,76.470588
35,0.068000,0.679513,82.352941


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Step,Training Loss,Validation Loss,Wer
5,1.178900,0.927745,67.647059
10,0.257400,0.724700,147.058824
15,0.249900,0.673844,67.647059
20,0.236100,0.703254,79.411765
25,0.100800,0.680304,76.470588
30,0.087000,0.674514,76.470588
35,0.068000,0.679513,82.352941
40,0.050300,0.664164,88.235294
45,0.061900,0.644890,91.176471
50,0.045500,0.632502,88.235294


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


[]

In [ ]:
# This script takes a Tamil audio file, translates and transcribes it to English text using Whisper, and prints out accuracy metrics.

from transformers import pipeline
from jiwer import wer
import time

# ---- Upload Tamil audio (for Colab, can modify for local use) ----
try:
    from google.colab import files
    print("Upload a .wav Tamil audio file:")
    uploaded = files.upload()
    tamil_path = list(uploaded.keys())[0]  # Get the filename of the uploaded file
except ImportError:
    # For non-Colab/local runs
    tamil_path = input("Enter path to your Tamil .wav file: ")

# ---- Get the ground-truth transcript (in English) ----
ground_truth_english = input("Paste the correct English translation for the audio: ").strip().lower()

# ---- Load Whisper pipeline for translation (Tamil-to-English) ----
print("Loading Whisper pipeline for Tamil-to-English translation...")
whisper_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base",  # You can use 'large' for better accuracy
    return_timestamps=True,
    #generate_kwargs={"task": "translate", "language": "ta"}
)

print("Transcribing and translating audio (Tamil → English)...")
start_time = time.time()
result = whisper_pipeline(tamil_path)
translated_text = result["text"].strip().lower()
end_time = time.time()
elapsed_time = end_time - start_time

# ---- Show results and WER metric ----
print("\n[Whisper Tamil → English]")
print("Transcription/Translation:", translated_text)
print("Ground Truth:", ground_truth_english)
print("Time:", round(elapsed_time, 2), "seconds")
if ground_truth_english:
    trans_wer = wer(ground_truth_english, translated_text)
    print("WER:", round(trans_wer, 2))
else:
    print("No ground truth provided; WER not computed.")

Upload a .wav Tamil audio file:


Saving tamilExtension.wav to tamilExtension (8).wav
Paste the correct English translation for the audio: "Let the same fire that burns him burn me too," he cried, wearily. The world of plants is not as bad as the human race, which has repeatedly picked itself up without falling down.
Loading Whisper pipeline for Tamil-to-English translation...


Device set to use cpu


Transcribing and translating audio (Tamil → English)...


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.



[Whisper Tamil → English]
Transcription/Translation: he is the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the son of the s

In [ ]:
#Testing Wav2Vec2 ASR MODEL
#getting model and tokenizer from HuggingFace repo
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model_w2v = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
#soundfile lib used for reading / writing to audo files
import torchaudio
#two values returned from read() : the audio data itself ( as array) and sample rate of audio file
speech, rate = torchaudio.load(audio_path)
# audio data returned as tensor, multi dim array in PyTorch and sample rate ( audio samples/ sec)
speech_tensor, rate = torchaudio.load(audio_path)
# removing dimensions from the tensor that have a size of 1
speech = torch.squeeze(speech_tensor, 0)
#converts pytorch tensor back into numpy array
speech = speech.numpy()

# need to resample audio rate to 16KHz if not 16KHz, since Wav2Vec2 expects input audio at a 16kHz sample rate
# can use torch audio or librosa ( used librosa elsewhere, wanted to use differnet mechanism here)
if rate != 16000:
  #creating ReSample object from torchaudio ( original rate and indicate wanted rate)
  resampler = torchaudio.transforms.Resample(orig_freq=rate, new_freq=16000)
  speech_tensor = resampler(speech_tensor) #changes sample rate
  #converting tensor back into numpy array, stored as speech
  speech = speech_tensor.squeeze().numpy()
  rate = 16000

#tokenize
# takes speech audio as input , "pt " tells tokenizer to return output as Pytorch tensors
#longest padding : pads audio arrays to same len ( if you wer proccesing multiple files at once, this would
# pad shorter seqs to len of longest one, not needed here since only one audio file; req arg)
input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values

#inference with the model - we dont need gradients ( not training, just inference); speeds up compute time, saves mem
with torch.no_grad():
  start = time.time()
  #pass audio input to model to get raw output scores, logits( the models initial predictions)
  #logits represent model confidence for each possible token ( chars or subwords) at each point in time in audio
  logits= model_w2v(input_values).logits
  end = time.time()
  w2v_time = end - start

  # argmax finds index of the maximum value along specified dim
  #dim = -1 , argmax is to be done on last dim of logits tensor,  in asr model output, the last dim
  # represents diff possible tokens model can predict. Goes through the logits for each point in time
  #and finds index of the token with highest score ( most likely token). result is sequence of indices showing the
  # most probable tokens predicted by model for audio input.
  predicted_ids = torch.argmax(logits, dim=-1)

  #decode list of predicted token ids into readable text
  #batch_decode converts list of token ids back into  text - just one result
  w2v_text_list = tokenizer.batch_decode(predicted_ids)
  if (len(w2v_text_list) > 0):
    #since i processed single file, list should have one transcribed string
    w2v_text = w2v_text_list[0]
  else :
    w2v_text = " "

#cleanup
w2v_text = w2v_text.strip().lower()
  #calc wer rate
w2v_wer = wer(ground_truth, w2v_text)
print("\n[Wav2Vec2]")
print("WER:", round(w2v_wer, 2))
print("Time:", round(w2v_time, 2), "seconds")
print("Transcription:", w2v_text)

#example of logits for my understanding
#at some moment, model "confidence scores" (logits) for three possible letters ('a', 'b', 'c') are:
# 'a' = 0.5, 'b' = 2.1 ,'c' = -1.0. here, logits for'b' (2.1) is the highest.
# we have torch.argmax([0.5, 2.1, -1.0], dim=-1) , result would be index of highest value which is 1
# in our Wav2Vec2  model, we have scores for many possible tokens ( entire vocab) and for many points in time across
# the audio. arg max picks most likely token at each point in time based on scores.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



[Wav2Vec2]
WER: 0.33
Time: 18.61 seconds
Transcription: hello my name is amrich i'm currently a fourth year studying computer science at north carolina state university two years ago in twenty twenty three a i and machine learning got superimportant intech health care and even education in this project i will be trying to prototype a s ar model that begins with english to english conversion then i will extend the model to support tamel to english conversion finally i will try serastra to enguish serastrians are a linguistic and cultural community in tamel nadu india originally from the serastr region of gugerat who speak the indoary and serastra language


In [ ]:
#Testing Vosk asr model
!wget -q https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip -q vosk-model-small-en-us-0.15.zip
!ls vosk-model-small-en-us-0.15
#import librosa to change sampling rate to supported rate for vosk
import librosa
#import soundfile for reading/writing audio files
import soundfile as sf
inputAudio = audio_path
#load data
data, sampleRate = sf.read(inputAudio)
#coverting to mono if needed
if len(data.shape) > 1:
  data = librosa.to_mono(data.T)
#resample to 16khz if needed
if sampleRate != 16000:
  data = librosa.resample(data, orig_sr=sampleRate, target_sr=16000)
#save as PCM 16 bit wav
sf.write("converted.wav", data, 16000, subtype='PCM_16')
audio_path = "converted.wav"
# path downloaded vosk model folder
vosk_model_path = "vosk-model-small-en-us-0.15"
#loads acc lang model from the said path that holds the model files into memory
vosk_model = VoskModel(vosk_model_path)
#open audio file for r : reading and  b means open in binary mode.
wf = wave.open(audio_path, "rb")
# you pass the loaded vosk_model into recognizer so it knows which lang model to use
#getnchannels checks if num of channels is not 1 ( not mono audio )  and getsamwidth checks if sample
# width ( bytyes / sample) is not 2. getcomptype checks if the compression type is not NONE
# vosk expects uncompressed audio data ( PCM fomrat)
if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
    print ("Audio file must be WAV format mono PCM")
    wf.close()
    voskText = " "
    voskTime = "N/A"
else :
  #init vosk recognizer( performs actual speech recognition) with model and sample rate from file
  vosk_recognizer = KaldiRecognizer(vosk_model, wf.getframerate()) # getframerate() as the second param gets sample rate of the audio.
  allText = [] # to keep up with chunks of recognized text
  #partialResults = []

  start = time.time()

  #read audio until the  end
  while True:
    #read chunk of audio data ( 4000 audio frames - frame has samples of channels at given point in time )
    data = wf.readframes(4000)
    if len(data) == 0:
      break # EOF
    #feeding data into recgonizer for processing in increments.
    #returns true if it has produced a partial result ready to be retreived,
    ok = vosk_recognizer.AcceptWaveform(data)
    if ok:
      #Result() returns json string that has partial transcription
      partialRes = vosk_recognizer.Result()
      print("Partial result:", partialRes)
      #extract text from the partial result
      try :
        text = json.loads(partialRes)["text"]
        if text :
          allText.append(text)
      except Exception as e:
        print("error in partial result",e)
        text = ""

  #add final recognition result
  # after all audio data processed, a json string having complete and final transcription is appended,
  # list of  partial results + final result all appended to be one string
  finalRes = vosk_recognizer.FinalResult()

  import json
  #extract text from  json result, attempts to add final result( can be empty since sometimes, finalResult() returns only what was not already included in the partials)
  #catches errors with the voskResult
  try :
    txt = json.loads(finalRes).get("text", "")
    if txt:
      allText.append(txt)
  except Exception as e:
    print("error in final result", e)

  end = time.time()
  voskTime = end - start

  #join recgonized chunks into one full transcript ( cleaned up )
  voskText = " ".join(allText)
  voskText = voskText.strip().lower()


#results of metrics
vosk_wer = wer(ground_truth, voskText)
print("\n[Vosk]")
print("WER:", round(vosk_wer, 2))
print("Time:", round(voskTime, 2), "seconds")
print("Transcription:", voskText)




replace vosk-model-small-en-us-0.15/am/final.mdl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/graph/disambig_tid.int? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/graph/HCLr.fst? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/graph/Gr.fst? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/graph/phones/word_boundary.int? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/conf/model.conf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/conf/mfcc.conf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/ivector/splice.conf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/ivector/final.dubm? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/ivector/global_cmvn.stats? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace vosk-model-small-en-us-0.15/ive

In [ ]:
#Testing Google speech to text api
from google.cloud import speech
#credentials.json
import os
import wave

#env variable that tells google libraries where to find service account credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/credentials.json"

# client that can talk with Google cloud speech to text api
client = speech.SpeechClient()
#audio file path ( mono , 16 bit 16000Hz WAV for best results)

#read audio data into mem, opens file in binary mode
with open(audio_path, "rb") as f:
    content = f.read()

# Get the sample rate from the WAV file header
try:
    with wave.open(audio_path, 'rb') as wf:
        audio_sample_rate = wf.getframerate()
except wave.Error as e:
    print(f"Error reading WAV file header: {e}")
    audio_sample_rate = 16000 # default to 16000 if reading fails

# set audio and recognition config
#tells google what audio you are sending , the audio data is wrapped in a RecognitionAudio object
audio = speech.RecognitionAudio(content=content)
# the configuration for recognition
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16, # PCM WAV encoding
    sample_rate_hertz= audio_sample_rate,
    language_code="en-US", # english
)
#call api
start = time.time()

#sends audio and config as a request to google api and waits for response, recognize() returns response object with transcripts
response = client.recognize(config=config, audio=audio)
end = time.time()
totalTime = end - start

#extract transcription
googleText = " "
#since google may return multiple results, we have to have a for loop where for each result
# we take the most likely and combine them
for result in response.results:
    #alternatives[0] is the top (most confident) transcription for this element as thought by the api , so the first alternative
    #transcript looks at acc text string of the transcription of that alt
    googleText += result.alternatives[0].transcript + " "
googleText = googleText.strip().lower()
google_wer = wer(ground_truth, googleText)
print("\n[Google Speech-to-Text]")
print("WER:", round(google_wer, 2))
print("Time:", round(totalTime, 2), "seconds")
print("Transcription:", googleText)

#now I have got audio to Text, so asking user toupload a .wav file and having a truth value
# then i ran my audio data through the google speech to text api model to get predictions. metrics down below
# Now, i want to sport live speech, so letting user speak and converting this to text in english



[Google Speech-to-Text]
WER: 0.23
Time: 9.12 seconds
Transcription: hello my name is amish  i'm currently a fourth-year studying computer science at north carolina state university  2 years ago in 2023 ai and machine learning got super important in tech healthcare and even education  in this project i will be trying to prototype a asr model that begins with english to english conversion  then i will extend the model to support tamil to english conversion  finally i will try to rational to english  sarians are a linguistic and cultural community in tamil nadu india originally from the sasha region of gujarat who speak the indo-aryan sastre language


In [ ]:
!pip install deep_translator

In [ ]:
#fine tune whisper , above I just tested with a small set of input audios
# i should upload a plethora of audio files in tamil to train and potentially tune, helping with domain or accent adaption
#creating a dataset with columns audio and transcript in tamil, allowing me to create parallel Tamil English datasets
# from this file, I can also populate hindi , english and eventually sourashtra translations of the audio
import pandas as pd
import os
from pydub import AudioSegment # audio conversion
from googletrans import Translator

#reading in the validated.tsv file ( this file has meta deta about the audio files. I have linked these audio files in the clips directory here)
# using the tsv file and its cols (the path and transcript ), I will make output df , so I can prepare the resulting dataframe for model training

df = pd.read_csv("validated.tsv", sep="\t")
if df is not None:
    # check if the 'clips' directory exists
    clips_dir = "clips"
    if not os.path.exists(clips_dir):
        print("directory not found")
    else:
        #output dir for wavs
        os.makedirs("wav_clips", exist_ok=True)
        #convert mp3 to wav , create lists for dataset, this will hold full paths and transcripts for each tamil audio
        audio_paths = []
        transcripts = [] # tamil
        engTransripts = [] # english
        hindiTranscripts = [] # hindi

        # loop will go through each entry in the tsv file and extract the path name
        for index, row in df.iterrows():
          mp3_file = os.path.join(clips_dir, row["path"])
          #construct input mp3 and output is wav
          wav_file = os.path.join("wav_clips", row["path"].replace(".mp3", ".wav"))

          #convert only if wav does not already exist
          if not os.path.exists(wav_file):
            try:
                #load the mp3 , set sample rate ( covert to mono) and export the file as wav
                audio = AudioSegment.from_file(mp3_file)
                audio = audio.set_frame_rate(16000).set_channels(1)  # 16kHz mono
                audio.export(wav_file, format="wav")
            except FileNotFoundError:
                 print("Mp3 file not found")
                 continue # skip this file and continue with the next
          # i am populating the audio path and transcripts list
          audio_paths.append(wav_file)
          #row will be dict ( keys represent the column name)and sentence key holds the sentence value for that particular audio file
          transcripts.append(row["sentence"])
          #translate to english
          translator = Translator()
          engTransripts.append(translator.translate(row["sentence"], src='ta', dest='en').text)
          translator = Translator()
          hindiTranscripts.append(translator.translate(row["sentence"], src='ta', dest='hi').text)

        #now we can build dataframe and save as dataset with tamil audio and its tamil translation as specified in validation.tsv
        dataset_df = pd.DataFrame({
            "audio": audio_paths,
            "transcript_Tamil": transcripts,
            "transcript_English": engTransripts,
            "transcript_Hindi": hindiTranscripts
        })

        dataset_df.to_csv("tamil_asr_dataset.csv", index=False)
        print("✅ Dataset saved as tamil_asr_dataset.csv")
        print(dataset_df.head())

AttributeError: 'coroutine' object has no attribute 'text'